In [1]:
# Dependencies
# ----------------------------------
import pandas as pd

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date, DateTime

from sqlalchemy import create_engine, ForeignKey

Base = declarative_base()

# from sqlalchemy.types import Boolean, Date, DateTime, Float, Integer, Text, Time, Interval

In [2]:
msa_df = pd.read_csv('../data/final_msa.csv', index_col=0)
msa_df.head()

,incident_id,incident_date,state,city_county,address,number_killed,number_injured,latitude,longitude
0,1993551,2021-05-03,Michigan,Saginaw,1315 Randolph St,1,4,39.939999,-75.225515
1,1993090,2021-05-02,Ohio,Columbus,491 Park St,0,4,41.750057,-87.548293
2,1993664,2021-05-02,Pennsylvania,Philadelphia,5200 block of Burton St,1,3,40.778605,-74.227081
3,1993221,2021-05-02,Maryland,Baltimore,1500 block of Washington Blvd,1,3,34.728035,-92.266020
4,1993280,2021-05-02,Illinois,Chicago,2100 block of W Adams St,0,5,35.005750,-89.959256


In [3]:
msa_df['incident_date'] =  pd.to_datetime(msa_df['incident_date'])

In [4]:
msa_df.dtypes

incident_id                int64
incident_date     datetime64[ns]
state                     object
city_county               object
address                   object
number_killed              int64
number_injured             int64
latitude                 float64
longitude                float64
dtype: object

In [5]:
# class Directors(Base):
#     __tablename__ = 'directors'
#     director_id = Column(String, primary_key=True)
#     director_name = Column(String(255))

In [6]:
# Create Classes
class Msa(Base):
    __tablename__ = 'msa'
    incident_id = Column(Integer, primary_key=True)
    incident_date = Column(Date)
    state = Column(String(255))
    city = Column(String(255))
    address = Column(String(255))
    killed = Column(Integer)
    injured = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)

In [7]:
# These tables only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'msa': Table('msa', MetaData(bind=None), Column('incident_id', Integer(), table=<msa>, primary_key=True, nullable=False), Column('incident_date', Date(), table=<msa>), Column('state', String(length=255), table=<msa>), Column('city', String(length=255), table=<msa>), Column('address', String(length=255), table=<msa>), Column('killed', Integer(), table=<msa>), Column('injured', Integer(), table=<msa>), Column('latitude', Float(), table=<msa>), Column('longitude', Float(), table=<msa>), schema=None)})

In [8]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///../data/msa.sqlite")
conn = engine.connect()

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [10]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
# incident_id	incident_date	state	city_county	address	number_killed	number_injured	latitude	longitude

#     incident_id = Column(Integer, primary_key=True)
#     incident_date = Column(Date)
#     state = Column(String(255))
#     city = Column(String(255))
#     address = Column(String(255))
#     killed = Column(Integer)
#     injured = Column(Integer)
#     latitude = Column(Float)
#     longitude = Column(Float)

In [12]:
# Note that adding to the session does not update the table. It queues up those queries.
# Adding data to directors table from directors csv file
for index, name in msa_df.iterrows():
    session.add(Msa(incident_id=name['incident_id'], incident_date=name['incident_date'], state=name['state'], \
                   city=name['city_county'], address=name['address'], killed=name['number_killed'], \
                   injured=name['number_injured'], latitude=name['latitude'], \
                   longitude=name['longitude']
                   ))

In [13]:
session.new

IdentitySet([<__main__.Msa object at 0x000001CFFB7C7438>, <__main__.Msa object at 0x000001CFFB7C7400>, <__main__.Msa object at 0x000001CFFB7C74A8>, <__main__.Msa object at 0x000001CFFB7C7518>, <__main__.Msa object at 0x000001CFFB7C7320>, <__main__.Msa object at 0x000001CFFB7C75C0>, <__main__.Msa object at 0x000001CFFB7C7630>, <__main__.Msa object at 0x000001CFFB7C76A0>, <__main__.Msa object at 0x000001CFFB7C7710>, <__main__.Msa object at 0x000001CFFB7C7780>, <__main__.Msa object at 0x000001CFFB7C77F0>, <__main__.Msa object at 0x000001CFFB7C7860>, <__main__.Msa object at 0x000001CFFB7C78D0>, <__main__.Msa object at 0x000001CFFBD96EB8>, <__main__.Msa object at 0x000001CFFB7C79B0>, <__main__.Msa object at 0x000001CFFB7C79E8>, <__main__.Msa object at 0x000001CFFB7C7A58>, <__main__.Msa object at 0x000001CFFB7C7AC8>, <__main__.Msa object at 0x000001CFFB7C7B38>, <__main__.Msa object at 0x000001CFFB7C7BA8>, <__main__.Msa object at 0x000001CFFB7C7C18>, <__main__.Msa object at 0x000001CFFB7C7C88

In [14]:
session.commit()

In [15]:
# session.rollback()

In [16]:
session.close()